In [27]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
SCORE_DIR = "data\\scores"

In [29]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [30]:
def parse_html(box_score):
    with open(box_score, encoding="utf8") as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [31]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [32]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [33]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [34]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 2596
200 / 2596
300 / 2596
400 / 2596
500 / 2596
600 / 2596
700 / 2596
800 / 2596
900 / 2596
1000 / 2596
1100 / 2596
1200 / 2596
1300 / 2596
1400 / 2596
1500 / 2596
1600 / 2596
1700 / 2596
1800 / 2596
1900 / 2596
2000 / 2596
2100 / 2596
2200 / 2596
2300 / 2596
2400 / 2596
2500 / 2596


ParserError: Document is empty

In [35]:
games_df = pd.concat(games, ignore_index=True)

In [36]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,99.0,0.374,10.0,33.0,0.303,15.0,23.0,...,43.6,40.4,300.0,98.0,BRK,125,1,2021,2020-12-22,False
1,240.0,240.0,42.0,92.0,0.457,15.0,35.0,0.429,26.0,32.0,...,50.0,32.1,267.0,120.0,GSW,99,0,2021,2020-12-22,True
2,240.0,240.0,44.0,93.0,0.473,14.0,40.0,0.350,14.0,19.0,...,100.0,35.9,166.0,118.0,LAL,109,1,2021,2020-12-22,True
3,240.0,240.0,38.0,81.0,0.469,9.0,29.0,0.310,24.0,31.0,...,20.9,40.2,154.0,114.0,LAC,116,0,2021,2020-12-22,False
4,240.0,240.0,46.0,90.0,0.511,14.0,35.0,0.400,15.0,18.0,...,33.3,39.2,203.0,126.0,BOS,122,1,2021,2020-12-23,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240.0,240.0,33.0,80.0,0.413,9.0,37.0,0.243,13.0,19.0,...,33.3,28.1,203.0,106.0,PHI,96,0,2023,2023-04-22,False
5186,240.0,240.0,41.0,84.0,0.488,9.0,22.0,0.409,21.0,27.0,...,29.1,35.8,140.0,128.0,LAC,100,1,2023,2023-04-22,True
5187,240.0,240.0,40.0,92.0,0.435,12.0,37.0,0.324,8.0,10.0,...,25.8,30.3,152.0,117.0,PHO,112,0,2023,2023-04-22,False
5188,240.0,240.0,38.0,85.0,0.447,15.0,39.0,0.385,8.0,12.0,...,37.5,37.3,236.0,113.0,MIA,121,1,2023,2023-04-22,False


In [ ]:
games_df.to_csv("nba_games.csv")